# Regression analysis

## Principle of regression

Supervised methods in ML includ classification (cf. titanic) and regression (cf. this exercise about house prices in Boston).

Regression = set of statistical methods helping us to find relationships between variables (predictors = independent variables; outcomes = dependent variables).

## Definition of the problem

The goal of the exercise is to predict the sale prices for each house (in Boston) based on the different features provided in the dataset. 

## Problematic



## Resolution approaches

## Parameters

## Import modules and load files